# Notebook az adatgyűjtésről.

Mivel az eredmények.com dinamikus html, így direktben nem fogjuk tudni megnyitni a BeautifulSoup csomaggal. Szükségünk lesz egy olyan csomagra, mely tudja ezeket kezelni, ehhez a Seleniumot fogjuk alkalmazni.

In [5]:
import selenium
import sys
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("E:\Ragács Attila\Documents\matek (egyetem)\Foci TDK\chromedriver_win32\chromedriver.exe", chrome_options=options) 

E:\Programok\Python3.7\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [7]:
driver.get("https://www.eredmenyek.com/foci/magyarorszag/otp-bank-liga-2017-2018/eredmenyek/")
more_buttons = driver.find_elements_by_class_name("event__more")
print(len(more_buttons))
for x in range(len(more_buttons)):
    if more_buttons[x].is_displayed():
        print(True)
        driver.execute_script("arguments[0].click();", more_buttons[x])
        time.sleep(1)
page_source = driver.page_source

1
True


Tulajdonképpen most annyit csináltunk, hogy megnyitottuk a weboldalt és a megkértük a programot, hogy nyomjon rá a További Meccsekre és az utána kapott oldalt olvassa be. Mivel a További Meccseket tartalmazó URL ugyanaz, mint az eredeti, a dinamikus html miatt, ezért van szükségünk Seleniumra. 

In [8]:
#page_source

In [9]:
soup = BeautifulSoup(page_source)

In [10]:
#soup

In [11]:
IDk=[]
for tag in soup.findAll('div',{'title':'A meccs részleteiért kattints ide!'}):
    IDk.append(tag.get('id'))

In [12]:
IDk[:5]

['g_1_UBAwg2oi',
 'g_1_829shM0c',
 'g_1_2P8oitG3',
 'g_1_nw4XZvoT',
 'g_1_r9k3UI8j']

In [13]:
IDk2=[]
for i in IDk:
    IDk2.append(i[4:len(i)+1])

In [14]:
IDk2[:5]

['UBAwg2oi', '829shM0c', '2P8oitG3', 'nw4XZvoT', 'r9k3UI8j']

IDk2-ben benne vannak a meccsek statisztikáinak eléréséhez szükséges "nevek".

In [15]:
Linkek=[]
for i in IDk2:
    Linkek.append('https://www.eredmenyek.com/merkozes/'+str(i)+"#a-merkozes-statisztikaja;0")

In [16]:
Linkek[0]

'https://www.eredmenyek.com/merkozes/UBAwg2oi#a-merkozes-statisztikaja;0'

Linkek is megvannak.

In [17]:
url=Linkek[0]
source = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(source)
#soup

In [18]:
soup.findAll('div',{'class':'statRow'})

[]

In [19]:
Dátum=[]
HCsapat=[]
ICsapat=[]
HGólok=[]
IGólok=[]
HLabdabirtoklás=[]
ILabdabirtoklás=[]
HKapuraLövés=[]
IKapuraLövés=[]
HKaputEltalálóLövések=[]
IKaputEltalálóLövések=[]
HKaputNEMTL=[]
IKaputNEMTL=[]
HSzabadrúgás=[]
ISzabadrúgás=[]
HSzöglet=[]
ISzöglet=[]
HBedobás=[]
IBedobás=[]
HVédés=[]
IVédés=[]
HOdds=[]
DOdds=[]
VOdds=[]

Hasonlóan dinamikusak a htmlek, így szükségünk van Seleniumra.

----------

A Ferencváros-Mezőkövesd (2020. 06. 27.) meccs oddsai 

In [20]:
link='https://www.eredmenyek.com/merkozes/8OAjGAer/#oddsok-osszehasonlitasa;over-under;rendes-j-i'
driver.get(link)
more_buttons = driver.find_elements_by_id("a-match-statistics")
for x in range(len(more_buttons)):
    if more_buttons[x].is_displayed():
        print(True)
        driver.execute_script("arguments[0].click();", more_buttons[x])
        time.sleep(10)
source = driver.page_source
soup = BeautifulSoup(source)
print(soup.findAll('span',{'class':'odds value'})[0].text.rstrip())
print(soup.findAll('span',{'class':'odds value'})[1].text.rstrip())
print(soup.findAll('span',{'class':'odds value'})[2].text.rstrip())

IndexError: list index out of range

In [4]:
def Adatok_letöltése(LINKEK):
    for i in range(len(LINKEK)):
        driver.get(LINKEK[i])
        more_buttons = driver.find_elements_by_id("a-match-statistics")
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        source = driver.page_source
        soup = BeautifulSoup(source)
        HStatisztikák=soup.findAll('div',{'class':'statText statText--homeValue'})
        IStatisztikák=soup.findAll('div',{'class':'statText statText--awayValue'})
        HVédés.append(HStatisztikák[7].string)
        IVédés.append(IStatisztikák[7].string)
        HOdds.append(soup.findAll('span',{'class':'odds value'})[0].text.rstrip())
        DOdds.append(soup.findAll('span',{'class':'odds value'})[1].text.rstrip())
        VOdds.append(soup.findAll('span',{'class':'odds value'})[2].text.rstrip())
        Dátum.append(soup.findAll('div',{'id':'utime'})[0].string)
        HCsapat.append(soup.findAll('a',{'class':'participant-imglink'})[1].string)
        ICsapat.append(soup.findAll('a',{'class':'participant-imglink'})[3].string)
        HGólok.append(soup.findAll('span',{'class':'scoreboard'})[0].string)
        IGólok.append(soup.findAll('span',{'class':'scoreboard'})[1].string)
        HLabdabirtoklás.append(HStatisztikák[0].string)
        ILabdabirtoklás.append(IStatisztikák[0].string)
        HKapuraLövés.append(HStatisztikák[1].string)
        IKapuraLövés.append(IStatisztikák[1].string)
        HKaputEltalálóLövések.append(HStatisztikák[2].string)
        IKaputEltalálóLövések.append(IStatisztikák[2].string)
        HKaputNEMTL.append(HStatisztikák[3].string)
        IKaputNEMTL.append(IStatisztikák[3].string)
        HSzabadrúgás.append(HStatisztikák[4].string)
        ISzabadrúgás.append(IStatisztikák[4].string)
        HSzöglet.append(HStatisztikák[5].string)
        ISzöglet.append(IStatisztikák[5].string)
        HBedobás.append(HStatisztikák[6].string)
        IBedobás.append(IStatisztikák[6].string)
		
		
#Futtatás:
#Értelemszerűen, ha elakad, akkor attól az n-től kezdve újra.

n=0
for i in Linkek:
    print(n)
    n+=1
    try: 
        Adatok_letöltése([i])
    except:
        try: 
            Adatok_letöltése([i])
        except:
            try:
                Adatok_letöltése([i])
            except:
                try:
                    Adatok_letöltése([i])
                except:
                        try:
                            Adatok_letöltése([i])
                        except:
                                try:
                                    Adatok_letöltése([i])
                                except: 
                                                try:
                                                    Adatok_letöltése([i])
                                                except:
                                                    try:
                                                        Adatok_letöltése([i])
                                                    except:
                                                        try:
                                                            Adatok_letöltése([i])
                                                        except:
                                                            try:
                                                                Adatok_letöltése([i])
                                                            except: 
                                                                print("Nem sikerült :(")
        

NameError: name 'Linkek' is not defined

Néha misztikus hibát dob, sejtésem szerint az oldal nem akar bejönni rendesen, vagy letilt vagy valami ilyesmi oka lehet. Emiatt azt csinálom, hogy amikor bármi hiba kerül bele, akkor megvizsgálom, hogy pontosan melyik mozzanatnál akadt el, és akkor onnantól indítom újra. Egyelőre ez nem okozott túl sok plusz munkát, de későbbiekben talán célszerű lenne implementálni a kódot, ami felismeri az elakadást és magától próbálkozik amíg nem sikerült.

----------

In [18]:
print(len(Dátum))
print(len(HCsapat))
print(len(ICsapat))
print(len(HGólok))
print(len(IGólok))
print(len(HLabdabirtoklás))
print(len(HBedobás))
print(len(HVédés))

198
198
198
198
198
198
198
253


In [42]:
Dátum=Dátum[:-1]
HCsapat=HCsapat[:-1]
ICsapat=ICsapat[:-1]
HGólok=HGólok[:-1]
IGólok=IGólok[:-1]
HOdds=HOdds[:-1]
VOdds=VOdds[:-1]

In [43]:
HCsapat

[]

In [44]:
Dátum

[]

In [45]:
HOdds

[]

In [39]:
Adatok_letöltése(Linkek[10:])

0
1


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=84.0.4147.105)


----------

In [17]:
df=pd.DataFrame(list(map(list,zip(*[Dátum,HCsapat,ICsapat,HGólok,IGólok,HLabdabirtoklás,ILabdabirtoklás,HKapuraLövés,IKapuraLövés,
HKaputEltalálóLövések,IKaputEltalálóLövések,HKaputNEMTL,IKaputNEMTL,HSzabadrúgás,ISzabadrúgás,HSzöglet,ISzöglet,
HBedobás,IBedobás,HVédés,IVédés,HOdds,DOdds,VOdds]))),
columns=["Dátum","HCsapat","ICsapat","HGólok","IGólok","HLabdabirtoklás","ILabdabirtoklás","HKapuraLövés","IKapuraLövés",
"HKaputEltalálóLövések","IKaputEltalálóLövések","HKaputNEMTL","IKaputNEMTL","HSzabadrúgás","ISzabadrúgás","HSzöglet","ISzöglet",
"HBedobás","IBedobás","HVédés","IVédés","HOdds","DOdds","VOdds"])

In [18]:
df

,Dátum,HCsapat,ICsapat,HGólok,IGólok,HLabdabirtoklás,ILabdabirtoklás,HKapuraLövés,IKapuraLövés,HKaputEltalálóLövések,...,ISzabadrúgás,HSzöglet,ISzöglet,HBedobás,IBedobás,HVédés,IVédés,HOdds,DOdds,VOdds
0,02.06.2018 19:00,Balmazújváros,Ferencvárosi TC,3,3,52%,48%,16,11,9,...,17,8,1,34,18,3,6,2.30,3.50,2.70
1,02.06.2018 19:00,Diósgyőri VTK,MOL Fehérvár FC,2,1,56%,44%,16,7,9,...,14,10,7,26,13,2,7,2.00,3.75,3.40
2,02.06.2018 19:00,Haladás,Mezőkövesd,1,0,47%,53%,9,4,5,...,19,4,4,16,20,2,4,1.90,3.50,4.00
3,02.06.2018 19:00,Honvéd FC,Vasas,3,1,58%,42%,12,8,7,...,21,2,3,25,15,4,4,2.04,3.60,3.40
4,02.06.2018 19:00,Puskás Akadémia,Paks,1,0,55%,45%,10,7,5,...,14,5,6,24,25,4,4,2.54,3.40,2.62
5,02.06.2018 19:00,Újpest,Debreceni VSC,2,1,55%,45%,13,7,8,...,13,7,5,15,26,4,4,1.80,3.75,4.50
6,27.05.2018 19:30,MOL Fehérvár FC,Honvéd FC,2,0,58%,42%,12,5,8,...,18,5,3,24,26,4,4,1.44,4.20,7.00
7,27.05.2018 17:30,Ferencvárosi TC,Diósgyőri VTK,4,0,54%,46%,14,3,12,...,18,9,3,22,23,2,6,1.22,5.75,13.00
8,27.05.2018 17:00,Debreceni VSC,Puskás Akadémia,1,1,45%,55%,14,9,9,...,22,2,5,26,17,0,6,1.75,3.50,4.75
9,27.05.2018 17:00,Mezőkövesd,Újpest,0,1,50%,50%,11,6,3,...,12,5,6,34,12,2,8,2.25,3.30,3.25


In [19]:
df.to_csv("OTPBankLigaAdatok1718_odds", index=False)

In [262]:
df2=pd.read_csv('OTPBankLigaAdatok1920')

In [263]:
df2

,Dátum,HCsapat,ICsapat,HGólok,IGólok,HLabdabirtoklás,ILabdabirtoklás,HKapuraLövés,IKapuraLövés,HKaputEltalálóLövések,...,HKaputNEMTL,IKaputNEMTL,HSzabadrúgás,ISzabadrúgás,HSzöglet,ISzöglet,HBedobás,IBedobás,HVédés,IVédés
0,27.06.2020 20:00,Ferencvárosi TC,Mezőkövesd,1,0,55%,45%,11,10,8,...,3,9,16,13,8,7,20,19,1,7
1,27.06.2020 20:00,Puskás Akadémia,Honvéd FC,2,1,44%,56%,8,7,4,...,4,5,9,18,7,3,24,15,1,2
2,27.06.2020 17:30,Debreceni VSC,Paks,1,1,51%,49%,8,5,4,...,4,1,12,18,9,4,29,25,3,3
3,27.06.2020 17:30,Diósgyőri VTK,Kisvárda,0,2,62%,38%,10,7,2,...,8,3,21,17,14,2,22,13,2,2
4,27.06.2020 17:30,Újpest,Kaposvár,5,0,51%,49%,16,7,11,...,5,5,14,11,2,4,19,18,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,04.08.2019 18:00,Diósgyőri VTK,Honvéd FC,2,1,48%,52%,10,11,5,...,5,6,14,15,6,11,20,18,4,3
194,04.08.2019 18:00,MOL Fehérvár FC,Kaposvár,4,2,65%,35%,13,5,9,...,4,3,19,7,9,3,21,12,0,5
195,03.08.2019 19:30,Kisvárda,Paks,1,0,39%,61%,9,6,3,...,6,3,9,17,8,7,24,30,3,2
196,03.08.2019 19:30,Mezőkövesd,Zalaegerszeg,1,0,46%,54%,11,10,4,...,7,7,13,19,5,5,17,29,3,3
